<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/files_2020/cce-color.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Pipeline de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Junio 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Pipeline de Datos de Contratación Pública**|
|**Descripción y alcance**|Notebook para la limpieza y validación de los datos de la base unificada.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

# Introducción

En este notebook, nos centraremos en la limpieza y preprocesamiento de los datos obtenidos de la base de contratos del Sistema Electrónico para la Contratación Pública (SECOP) I y II de Colombia. Los conjuntos de datos con los que estamos trabajando contienen una variedad de información sobre contratos públicos, desde sus detalles operativos hasta sus atributos financieros.

El objetivo principal de este ejercicio es preparar estos conjuntos de datos para análisis posteriores. Esta preparación incluye una serie de pasos que asegurarán que los datos estén en el formato correcto y sean consistentes, comprensibles y confiables.

Cargamos la base previamente descargada y con las variables unificadas entre los conjuntos de datos de cada fuente para asegurar que la misma información se represente de manera coherente en los mismos. Esto puede incluir el mapeo de categorías equivalentes entre los conjuntos de datos y la armonización entre variables y sus nombres.

A continuación, estandarizaremos los datos para garantizar que todas las variables se presenten en un formato uniforme. Este proceso puede involucrar la normalización de texto, la conversión de datos categóricos en formatos numéricos y la manipulación de fechas y tiempos para que se presenten de manera consistente.

Una parte crucial de este trabajo implica limpiar los datos de elementos no deseados. Esto implica quitar espacios extra, eliminar caracteres especiales y, en general, asegurarnos de que los datos sean lo más limpios y precisos posible.

Además, verificaremos los tipos de variables en nuestros conjuntos de datos para asegurarnos de que sean apropiados para el tipo de datos que contienen. Esto podría implicar la conversión de datos numéricos almacenados como texto en números reales, o viceversa.

Comprobaremos si hay registros duplicados en nuestros conjuntos de datos. Si existen, los almacenaremos para realizar la validación y evitar cualquier sesgo o inexactitud en los análisis posteriores.

Finalmente, validaremos la cantidad de valores nulos y los valores negativos en la variable de cantidad de contrato. En el caso de los valores nulos, decidiremos si tienen coherencia, dependiendo del contexto. Los valores negativos, si no tienen sentido en el contexto de nuestro análisis, serán almacenados y validados desde la descarga de los datos.

Al final de este proceso, tendremos un conjunto de datos limpio y de alta calidad que estará listo para ser utilizado en futuros análisis y modelado de datos.

## Cargue del archivo parquet

### Cargue de librerías

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.

In [1]:
### Paquetes usados para la exploración de datos

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import unidecode
import libreria.data_management as dm
# from dataprep.clean import clean_duplication
import warnings
warnings.filterwarnings('ignore')

In [2]:
years = ['2020', '2021', '2022', '2023']
SECOP = {}

for year in years:
    SECOP[year] = pd.read_csv(f'../../../muestras de datos/procesados/bronce/SECOP_{year}.csv', sep=';')

Validación tipos de variables

In [3]:
def clean_dates(date):
    if pd.isna(date):
        return None  # Si la fecha es nula, devuelve nulo
    try:
        return pd.to_datetime(date, errors='raise').strftime('%d-%m-%Y')
    except (ValueError, TypeError):
        return None

In [4]:
for year in years:
    SECOP[year]['ID_Contrato'] = SECOP[year]['ID_Contrato'].astype(str)
    SECOP[year]['ID_Proceso'] = SECOP[year]['ID_Proceso'].astype(str)
    SECOP[year]['Nombre_Entidad'] = SECOP[year]['Nombre_Entidad'].astype(str)
    SECOP[year]['Orden_Entidad'] = SECOP[year]['Orden_Entidad'].astype(str)
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].astype(str)
    SECOP[year]['Estado'] = SECOP[year]['Estado'].astype(str)
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].astype(str)
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].astype(str)
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].astype(str)
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].astype(str)
    SECOP[year]['Documento_Proveedor'] = SECOP[year]['Documento_Proveedor'].astype(str)
    SECOP[year]['Tipo_proveedor'] = SECOP[year]['Tipo_proveedor'].astype(str)
    SECOP[year]['Departamento_Entidad'] = SECOP[year]['Departamento_Entidad'].astype(str)
    SECOP[year]['Municipio_Entidad'] = SECOP[year]['Municipio_Entidad'].astype(str)
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].astype(str)
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].astype(str)
    SECOP[year]['Link'] = SECOP[year]['Link'].astype(str)

    SECOP[year]['Valor_contrato'] = SECOP[year]['Valor_contrato'].astype('int64')

    SECOP[year]['Fecha_firma'] = SECOP[year]['Fecha_firma'].apply(clean_dates)
    SECOP[year]['Fecha_inicio_contrato'] = SECOP[year]['Fecha_inicio_contrato'].apply(clean_dates)
    SECOP[year]['Fecha_fin_contrato'] = SECOP[year]['Fecha_fin_contrato'].apply(clean_dates)

In [5]:
# try:
#     SECOP[year]['Fecha_fin_contrato'] = pd.to_datetime(SECOP[year]['Fecha_fin_contrato'], format='%d-%m-%Y', errors='raise')
# except ValueError as e:
#     # La conversión falló, por lo que algunos valores no tienen el formato esperado.
#     print("Error de conversión:", e)
#     # Filtramos los valores que no tienen el formato deseado y los guardamos en un nuevo DataFrame.
#     valores_no_validos = SECOP[year][~SECOP[year]['Fecha_fin_contrato'].notna()]
#     # Muestra los valores que no tienen el formato deseado
#     print("Valores no válidos:")
#     print(valores_no_validos)

In [6]:
def replace_nan(value):
    if value == 'nan':
        return ''
    return value

In [7]:
for year in years:
    SECOP[year]['ID_Contrato'] = SECOP[year]['ID_Contrato'].apply(replace_nan)
    SECOP[year]['ID_Proceso'] = SECOP[year]['ID_Proceso'].apply(replace_nan)
    SECOP[year]['Nombre_Entidad'] = SECOP[year]['Nombre_Entidad'].apply(replace_nan)
    SECOP[year]['Orden_Entidad'] = SECOP[year]['Orden_Entidad'].apply(replace_nan)
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].apply(replace_nan)
    SECOP[year]['Estado'] = SECOP[year]['Estado'].apply(replace_nan)
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].apply(replace_nan)
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].apply(replace_nan)
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].apply(replace_nan)
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].apply(replace_nan)
    SECOP[year]['Documento_Proveedor'] = SECOP[year]['Documento_Proveedor'].apply(replace_nan)
    SECOP[year]['Tipo_proveedor'] = SECOP[year]['Tipo_proveedor'].apply(replace_nan)
    SECOP[year]['Departamento_Entidad'] = SECOP[year]['Departamento_Entidad'].apply(replace_nan)
    SECOP[year]['Municipio_Entidad'] = SECOP[year]['Municipio_Entidad'].apply(replace_nan)
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].apply(replace_nan)
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].apply(replace_nan)
    SECOP[year]['Link'] = SECOP[year]['Link'].apply(replace_nan)

Tratamiento de espacios y caracteres especiales

In [8]:
for year in years:
    SECOP[year]['Nombre_Entidad'] = SECOP[year]['Nombre_Entidad'].apply(dm.remove_extra_punct)
    SECOP[year]['Orden_Entidad'] = SECOP[year]['Orden_Entidad'].apply(dm.remove_extra_punct)
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].apply(dm.remove_extra_punct)
    SECOP[year]['Estado'] = SECOP[year]['Estado'].apply(dm.remove_extra_punct)
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].apply(dm.remove_extra_punct)
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].apply(dm.remove_extra_punct)
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].apply(dm.remove_extra_punct) 
    SECOP[year]['Departamento_Entidad'] = SECOP[year]['Departamento_Entidad'].apply(dm.remove_extra_punct)
    SECOP[year]['Municipio_Entidad'] = SECOP[year]['Municipio_Entidad'].apply(dm.remove_extra_punct)
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].apply(dm.remove_extra_punct)
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].apply(dm.remove_extra_punct)

Tratamiento de espacios con strip

In [9]:
for year in years:
    SECOP[year]['ID_Contrato'] = SECOP[year]['ID_Contrato'].str.strip()
    SECOP[year]['ID_Proceso'] = SECOP[year]['ID_Proceso'].str.strip()
    SECOP[year]['Nombre_Entidad'] = SECOP[year]['Nombre_Entidad'].str.strip()
    SECOP[year]['Orden_Entidad'] = SECOP[year]['Orden_Entidad'].str.strip()
    SECOP[year]['Modalidad'] = SECOP[year]['Modalidad'].str.strip()
    SECOP[year]['Estado'] = SECOP[year]['Estado'].str.strip()
    SECOP[year]['Descripcion_proceso'] = SECOP[year]['Descripcion_proceso'].str.strip()
    SECOP[year]['Objeto_Contrato'] = SECOP[year]['Objeto_Contrato'].str.strip()
    SECOP[year]['Nombre_Proveedor'] = SECOP[year]['Nombre_Proveedor'].str.strip()
    SECOP[year]['Departamento_Entidad'] = SECOP[year]['Departamento_Entidad'].str.strip()
    SECOP[year]['Municipio_Entidad'] = SECOP[year]['Municipio_Entidad'].str.strip()
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].str.strip()
    SECOP[year]['Municipio_Proveedor'] = SECOP[year]['Municipio_Proveedor'].str.strip()

Estandarización de las modalidades

In [10]:
for year in years:
    SECOP[year]["Modalidad Estandar"] = SECOP[year]['Modalidad'].apply(dm.Tipo_Proceso)
    SECOP[year]["Grupo Modalidad"] = SECOP[year]["Modalidad Estandar"].apply(dm.Grupo_Modalidad)

Arreglo sobre estado

In [11]:
estado_contractual = {
    'modificado': 'Modificado',
    'celebrado': 'Celebrado',
    'terminado': 'Terminado',
    'liquidado': 'Liquidado',
    'cerrado': 'Cerrado',
    'activo': 'Activo',
    'cedido': 'Cedido',
    'suspendido': 'Suspendido',
    'convocado': 'Convocado',
    'terminado liquidar': 'Terminado liquidar',
    'terminado anormalmente despues convocado': 'Terminado anormalmente despues convocado',
    'terminado': 'Terminado',
    'ejecución': 'En ejecución',
    'cancelado': 'Cancelado',
    'enviado proveedor': 'Enviado proveedor'
}

for year in years:
    SECOP[year]['Estado'] = SECOP[year]['Estado'].replace(estado_contractual)

Arreglo sobre UNSPSC:

1. Separación por segmento, familia, clase y producto

In [12]:
for year in years:
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('V1', '')
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('.0', '')
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('UNSPECIFIED', '')
    SECOP[year]['UNSPSC'] = SECOP[year]['UNSPSC'].str.replace('.', '')
    if SECOP[year]['UNSPSC'].str != '':
        SECOP[year]['SEGMENTO_UNSPSC'] = SECOP[year]['UNSPSC'].str[:2]
        SECOP[year]['FAMILIA_UNSPSC'] = SECOP[year]['UNSPSC'].str[2:4]
        SECOP[year]['CLASE_UNSPSC'] = SECOP[year]['UNSPSC'].str[4:6]
        SECOP[year]['PRODUCTO_UNSPSC'] = SECOP[year]['UNSPSC'].str[6:]

Arreglo sobre Departamento_Entidad y Departamento_Proveedor

In [13]:
for year in years:
    SECOP[year]['Departamento_Entidad'] = SECOP[year]['Departamento_Entidad'].str.replace('distrito capital bogota', 'bogota d.c.')
    SECOP[year]['Departamento_Entidad'] = SECOP[year]['Departamento_Entidad'].str.replace('bogota d.c .', 'bogota d.c.')
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].str.replace('bogota d.c .', 'bogota d.c.')
    SECOP[year]['Departamento_Proveedor'] = SECOP[year]['Departamento_Proveedor'].str.replace('distrito capital bogota', 'bogota d.c.')

### Próximos requerimientos

- Cruce con la versión anterior del maestro de entidades (campos ```ID_Entidad, Nombre_Entidad, NIT_Entidad```)
- Actualización del maestro de entidades
- Limpieza del campo NIT_Entidad (digito de verificación)
- Arreglo sobre Documento_Proveedor
- Cruce de departamentos y municipios con el código divipola

### Validaciones sobre la versión actual

* Verificar duplicados en contratos

In [14]:
SECOP[year]['ID_Contrato'].value_counts().reset_index().head(10)

,index,ID_Contrato
0,9508689,58
1,11630933,50
2,10216031,46
3,9769383,45
4,10198728,40
5,9907288,40
6,10582940,39
7,10024941,35
8,9647601,33
9,10352894,32


In [ ]:
duplicados_2023 = SECOP['2023']['ID_Contrato'].value_counts().reset_index()

In [ ]:
print('Cantidad de contratos duplicados:')
print(len(duplicados_2023[(duplicados_2023['ID_Contrato']>1)]))
duplicados_2023[(duplicados_2023['ID_Contrato']>1)].head(10)

Cantidad de contratos duplicados:
56002


,index,ID_Contrato
0,12653737,40
1,12539047,40
2,12508380,40
3,12539128,40
4,12559493,40
5,12479188,40
6,12643080,40
7,12527785,40
8,12549871,40
9,12582097,30


Detectar negativos en valor del contrato

In [ ]:
for year in years:
    print('Cantidad de contratos con valor negativo 2023:', len(SECOP['2023'][(SECOP['2023']['Valor_contrato']<0)]))
    print('ID de los contratos 2023:', SECOP['2023'][(SECOP['2023']['Valor_contrato']<0)]['ID_Contrato'].unique())

Cantidad de contratos con valor negativo 2023: 0
ID de los contratos 2023: []


Se unifican las varibles departamento_entidad, municipio_entidad, departamento_proveedor y municipio_proveedor

In [15]:
columnas_geograficas = ['Departamento_Entidad','Departamento_Proveedor', 'Municipio_Entidad', 'Municipio_Proveedor']

for column in columnas_geograficas:
    for year in years:
        SECOP[year][column] = SECOP[year][column].str.upper()
        # Se llenan los campos vacíos con "No Definido"
        SECOP[year][column] = SECOP[year][column].fillna("No Definido")

Se ven los primeros registros del dataframe

In [ ]:
SECOP['2023'].head(5)

,index,ID_Contrato,ID_Proceso,ID_Entidad,Nombre_Entidad,NIT_Entidad,Orden_Entidad,Modalidad,Estado,Descripcion_proceso,Objeto_Contrato,Tipo_de_contrato,Fecha_firma,UNSPSC,Nombre_Proveedor,Documento_Proveedor,Tipo_proveedor,Valor_contrato,Departamento_Entidad,Municipio_Entidad,Departamento_Proveedor,Municipio_Proveedor,Fecha_inicio_contrato,Fecha_fin_contrato,Link,Fuente,Año_firma,Modalidad Estandar,Grupo Modalidad,SEGMENTO_UNSPSC,FAMILIA_UNSPSC,CLASE_UNSPSC,PRODUCTO_UNSPSC
0,2997478,12056146,22-4-13019919,21523847.0,boyaca empresa servicios publicos domiciliario...,891855578,territorial,regimen especial,Celebrado,construccion box culvert carrera quebrada aroma,construccion box culvert carrera quebrada aroma,Obra,03-03-2023,721412,ingtec sas,826000391,826000391,224502972,BOYACA,DUITAMA,BOYACA,BOYACA,03-03-2023,03-06-2023,https://www.contratos.gov.co/consultas/detalle...,SECOP I,2023,Régimen Especial,Modalidad Especial,72,14,12,
1,2997479,12056146,22-4-13019919,21523847.0,boyaca empresa servicios publicos domiciliario...,891855578,territorial,regimen especial,Celebrado,construccion box culvert carrera quebrada aroma,construccion box culvert carrera quebrada aroma,Obra,03-03-2023,721412,ingtec sas,826000391,826000391,224502972,BOYACA,DUITAMA,BOYACA,BOYACA,03-03-2023,03-06-2023,https://www.contratos.gov.co/consultas/detalle...,SECOP I,2023,Régimen Especial,Modalidad Especial,72,14,12,
2,3034475,12139930,22-4-13072912,21523847.0,boyaca empresa servicios publicos domiciliario...,891855578,territorial,regimen especial,Celebrado,interventora tcnica administrativa financiera ...,interventora tcnica administrativa financiera ...,Consultoría,03-03-2023,8115,edison fabian velandia rodriguez,1118547359,1118547359,12682782,BOYACA,DUITAMA,BOYACA,BOYACA,03-03-2023,03-06-2023,https://www.contratos.gov.co/consultas/detalle...,SECOP I,2023,Régimen Especial,Modalidad Especial,81,15,,
3,3034476,12139930,22-4-13072912,21523847.0,boyaca empresa servicios publicos domiciliario...,891855578,territorial,regimen especial,Celebrado,interventora tcnica administrativa financiera ...,interventora tcnica administrativa financiera ...,Consultoría,03-03-2023,8115,edison fabian velandia rodriguez,1118547359,1118547359,12682782,BOYACA,DUITAMA,BOYACA,BOYACA,03-03-2023,03-06-2023,https://www.contratos.gov.co/consultas/detalle...,SECOP I,2023,Régimen Especial,Modalidad Especial,81,15,,
4,3116094,12585001,18-12-8419746,205001178.0,antioquia agencia gestion paisaje patrimonio a...,900623766,territorial,contratacion directa ( ley ),Celebrado,presente contrato corresponde marco general vi...,servicio comunicaciones seguridad perimetral e...,Prestación de Servicios,27-03-2023,811118,une epm telecomunicaciones sa,900092385,900092385,59099638,ANTIOQUIA,MEDELLIN,ANTIOQUIA,ANTIOQUIA,07-04-2023,07-12-2023,https://www.contratos.gov.co/consultas/detalle...,SECOP I,2023,Contratación Directa,Modalidad Directa,81,11,18,


Almacenamiento del archivo como CSV y como parquet

In [16]:
for year in years:
    SECOP[year].to_csv(f'../../../muestras de datos/procesados/silver/SECOP_{year}.csv', index=False, sep=';')

In [ ]:
# Se revisa cantidad de nulos de cada variable

# SECOP['2023'].isnull().sum()

index                         0
ID_Contrato                   0
ID_Proceso                    0
ID_Entidad                  198
Nombre_Entidad                0
NIT_Entidad               17218
Orden_Entidad                 0
Modalidad                     0
Estado                        0
Descripcion_proceso           0
Objeto_Contrato               0
Tipo_de_contrato             18
Fecha_firma                   0
UNSPSC                        0
Nombre_Proveedor              0
Documento_Proveedor           0
Tipo_proveedor                0
Valor_contrato                0
Departamento_Entidad          0
Municipio_Entidad             0
Departamento_Proveedor        0
Municipio_Proveedor           0
Fecha_inicio_contrato     35167
Fecha_fin_contrato            1
Link                          0
Fuente                        0
Año_firma                     0
Modalidad Estandar            0
Grupo Modalidad               0
SEGMENTO_UNSPSC               0
FAMILIA_UNSPSC                0
CLASE_UN

# Conclusiones

Se realiza la limpieza de la base maestra, logrando el objetivo de tener un conjunto de datos limpio y de alta calidad que estará listo para ser utilizado en futuros análisis.

Se debe tener presente la alerta de valores duplicados en ID_Contrato, valores negativos en valor del contrato y valores nulos en variables donde se debe contar con la información (si los hay) para realizar las validaciones correspondientes.

El proceso se realiza con archivos descargados por año para agilizar la ejecución del notebook.

Finalmente, es importante recordar que la limpieza y el preprocesamiento de datos es un paso fundamental en cualquier proyecto de análisis de datos. La atención al detalle en esta etapa puede tener un impacto significativo en la calidad y la precisión de cualquier insight o modelo que se desarrolle en las etapas posteriores.